In [1]:
%reload_ext autoreload
%matplotlib inline

In [2]:
import sys
sys.path.append('../../Utils/')

In [3]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import nltk
import math
import random
from sklearn.svm import SVC
from sklearn.utils import shuffle
from load_imdb_data import load_imdb_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
import matplotlib.pyplot as plt 
np.random.seed(0)

In [4]:
data = open("../../data/intent_data.txt").readlines()

In [5]:
data = data[1:]

In [6]:
sentence = []
labels = []

for item in data:
    temp = item.split('   ')
    sentence.append(temp[0].strip())
    labels.append(temp[1].strip().replace("\n", ''))

In [16]:
#sentence = [line.decode('utf-8').strip() for line in sentence]
sentence = [line.strip() for line in sentence]

In [17]:
df = pd.DataFrame()
df['review'] = sentence
df['label'] = labels

In [18]:
df = df[df['label'].isin(['food', 'travel', 'career'])]

In [19]:
df.reset_index(drop=True, inplace=True)

In [20]:
df.head()

,review,label
0,I really want a hot dog . My co-workers went t...,food
1,"I wanna go to the beach , but ... Where is the...",travel
2,I really look forward to Egypt next week ! Its...,travel
3,I should buy more oatmeal cookies .. I just at...,food
4,"I want pancakes , should I make some ???",food


In [23]:
def processDocs(documents, vocab_size=5000):
    """
    This functions takes in a collection of documents and generates a vocabulary based on the size given in input. 
    It returns a representation for each document in the list of input documents. 
    """
    vocab = {} 
    doc_id = 0 
    doc_ids = []
    
    for doc in documents:
        doc_ids.append(doc_id)                          # Give an ID to each document 
        doc_id += 1
        
        for word in nltk.word_tokenize(doc):            # Generate a vocabulary while iterating threw the documents 
            if word not in vocab:
                vocab[word] = 1 
            else:
                vocab[word] += 1
    
    # Extract the most frequent words based on the vocabulary size 
    freq_words_list = sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:vocab_size]
    freq_words_set = set([item[0] for item in freq_words_list])
    
    # Give an index to each word in vocabulary 
    word2idx = {}         
    index_word = 0
    for word in freq_words_set:
        word2idx[word] = index_word
        index_word += 1
    word2idx['UNK'] = index_word
    
    doc_repr = []                          # Represent each document with representation based on the vocabulary  
    for doc in documents:
        temp = []
        for w in doc:
            if w in word2idx:
                temp.append(word2idx[w])
            else:
                temp.append(word2idx['UNK'])
        doc_repr.append(temp)
        
    return documents, doc_ids, word2idx, doc_repr

In [24]:
docs, doc_ids, word2ids, doc_repr = processDocs(df['review'], vocab_size=5000)

In [25]:
print(len(docs), len(doc_ids), len(word2ids), len(doc_repr))

591 591 1901 591


## Architecture - PV-DBOW Distributed Bag of Words version of Paragraph Vector

### Adding batchsizes for speedup

In [26]:
bucket_list = []

def generate_batch_pvdbow(doc_ids, doc_repr, sample_size=20, batch_size=1000, window_size=7):
    global bucket_list

    docs_ids_to_select = list(set(doc_ids) - set(bucket_list))
    
    
    if len(docs_ids_to_select) < batch_size//sample_size:
        bucket_list = []
        docs_ids_to_select = doc_ids
        
    index = 0 
    train_dX = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    train_label = np.ndarray(shape=(batch_size, window_size), dtype=np.int32)
    random_docs = random.sample(docs_ids_to_select, batch_size//sample_size)    # Choose set of random documents 

    bucket_list += random_docs
    
    for id_ in random_docs:
        for j in range(sample_size):                                 # Generating a dataset of sample size 
            random_index = random.randint(0, len(doc_repr[id_]) - window_size)
            sample_window = doc_repr[id_][random_index: random_index + window_size]
            train_dX[index] = id_
            train_label[index] = sample_window
            index += 1
    return train_dX, train_label 

In [27]:
doc_size = len(docs)
embedding_size_d = 100
embedding_size_w = 100
vocab_size = len(word2ids)
window_size = 7
n_neg_samples = 20
learning_rate = 10e-2
epochs = 10001
batch_size=1000
mu=0.99

In [28]:
# Define placeholders for training 
train_dX = tf.placeholder(tf.int32, shape=[batch_size, 1])
train_label = tf.placeholder(tf.int32, shape=[batch_size, window_size])

In [29]:
doc_embedding_np = np.random.randn(doc_size, embedding_size_d)/np.sqrt(doc_size + embedding_size_d)
word_embedding_np = np.random.randn(vocab_size, embedding_size_w)/np.sqrt(vocab_size + embedding_size_w)

In [30]:
# Define matrix for doc_embedding and word_embedding 
doc_embedding = tf.Variable(doc_embedding_np.astype(np.float32), name="doc_embedding")
word_embedding = tf.Variable(word_embedding_np.astype(np.float32),name="word_embedding")

In [31]:
# Define weights for the output unit 
weights = tf.Variable(tf.truncated_normal([vocab_size, embedding_size_d], 
                                       stddev=1.0 / math.sqrt(vocab_size)))
biases = tf.Variable(tf.zeros(vocab_size))

In [33]:
print (weights.get_shape())

(1901, 100)


In [34]:
# Add the doc2vec from the doc_embedding 
embed = tf.nn.embedding_lookup(doc_embedding, train_dX[:, 0])

In [35]:
loss = tf.nn.sampled_softmax_loss(weights=weights, \
                                  biases=biases, \
                                  labels=train_label, \
                                  inputs=embed, \
                                  num_sampled=n_neg_samples, \
                                  num_classes=vocab_size, \
                                  num_true=window_size)

In [36]:
loss = tf.reduce_mean(loss)

In [37]:
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

In [38]:
import os
os.system("mkdir /dev/shm/tensorflow_models")

0

In [44]:
saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    average_loss = 0
    
    for step in range(epochs):
        epoch_error = 0.0
        temp_dX, temp_labels = generate_batch_pvdbow(doc_ids=doc_ids, doc_repr=doc_repr)
        feed_dict = {train_dX : temp_dX,train_label : temp_labels}
        op, l = sess.run([optimizer, loss], 
                                    feed_dict=feed_dict)
        
        epoch_error += l
                
        if step % 1000 == 0:
            print ("Error at epoch : ", step, " = ", epoch_error)
            
    save_path = saver.save(sess, "/dev/shm/tensorflow_models/model_pvdbow_batch_training.ckpt")
    print("Model saved in file: %s" % save_path)

Error at epoch :  0  =  2.773125648498535
Error at epoch :  1000  =  2.086777448654175
Error at epoch :  2000  =  2.0167131423950195
Error at epoch :  3000  =  1.9949673414230347
Error at epoch :  4000  =  2.0192675590515137
Error at epoch :  5000  =  2.0004444122314453
Error at epoch :  6000  =  2.041379690170288
Error at epoch :  7000  =  1.977998971939087
Error at epoch :  8000  =  1.997300386428833
Error at epoch :  9000  =  2.0414466857910156
Error at epoch :  10000  =  1.9742205142974854
Model saved in file: /dev/shm/tensorflow_models/model_pvdbow_batch_training.ckpt


### Evaluation of the representation 

In [45]:
doc_pvdbow = None

with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "/dev/shm/tensorflow_models/model_pvdbow_batch_training.ckpt")
    print("Model restored.")
    doc2vec = doc_pvdbow = doc_embedding.eval()
    #performanceTest(doc2vec, list(imdb_data['sentiment']), method=None)

INFO:tensorflow:Restoring parameters from /dev/shm/tensorflow_models/model_pvdbow_batch_training.ckpt
Model restored.


In [46]:
from sklearn.manifold import TSNE
model = TSNE(perplexity=50, n_iter=5000)
Z = model.fit_transform(doc_pvdbow) 

In [47]:
df['axis1'] = Z[:, 0]
df['axis2'] = Z[:, 1]

In [48]:
df.reset_index(drop=True, inplace=True)

In [49]:
df.head()

,review,label,axis1,axis2
0,I really want a hot dog . My co-workers went t...,food,-0.686677,-1.011223
1,"I wanna go to the beach , but ... Where is the...",travel,0.963187,-1.709334
2,I really look forward to Egypt next week ! Its...,travel,1.409287,-1.347921
3,I should buy more oatmeal cookies .. I just at...,food,-1.954101,-0.941361
4,"I want pancakes , should I make some ???",food,0.866388,1.510582


In [51]:
from ggplot import *
ggplot(aes(x='axis1', y='axis2', color='label'), data=df)  + geom_point()

/home/piyush/Piyush/study/anthill/anthill/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/piyush/Piyush/study/anthill/anthill/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp


ImportError: cannot import name 'Timestamp'